In [2]:
import os
os.environ["NAPARI_ASYNC"] = "1"
#os.environ["NAPARI_PERFMON"] = "/Users/fukai/projects/napari-travali2/examples/prefmon.json"

In [3]:
import pandas as pd
import tensorstore as ts
from copy import deepcopy
spec = {
    "driver": "zarr3",
    "kvstore": {
            "driver": "file",
            "path": "/Volumes/Extreme SSD/aligned_image5_labels.zarr"
    },
    'context': {
            'cache_pool': {
                'total_bytes_limit': 100_000_000
            }
        },
        'recheck_cached_data':False,
    }
labels = ts.open(spec).result()[:,0,:,:]
image_spec = deepcopy(spec)
image_spec['kvstore']['path'] = '/Volumes/Extreme SSD/aligned_image5_image.zarr'
image = ts.open(image_spec).result()[:,0,:,:]
bboxes_df = pd.read_csv('/Volumes/Extreme SSD/aligned_image5_bboxes_df.csv').set_index(["frame","label"])
bboxes_df.head()

min_y  min_x  max_y  max_x
frame label                            
0     1         13   1865     57   1909
      2         19   4954     60   4994
      3         29   2832     70   2876
      4         35   1633     67   1669
      5         38   2756     80   2797

In [4]:
import numpy as np
import tensorstore as ts
import tensorstore_trackarr as tta
import json

print(labels.shape)
split_dict = {}
ta = tta.TrackArray(labels, split_dict, {}, bboxes_df)

(1965, 19991, 15437)


In [5]:
from napari import Viewer
from napari_travali2._stateful_widget import StateMachineWidget
from napari_travali2._logging import logger
import numpy as np

logger.setLevel("DEBUG")
logger.info("Starting napari-travali2")

viewer = Viewer()
viewer.add_image(image[:,5000:7000,5000:7000], translate=(0,5000,5000))
#viewer.add_image(image[:10])
labels_layer = viewer.add_labels(ta.array[:,5000:7000,5000:7000], translate=(0,5000,5000))
#labels_layer = viewer.add_labels(labels[:10])
labels_layer.editable=False

In [6]:
widget = StateMachineWidget(viewer, labels_layer, ta, multiscale=False)
viewer.window.add_dock_widget(widget, area="right")

/Users/fukai/projects/napari-travali2/.venv/lib/python3.13/site-packages/napari/_vispy/layers/scalar_field.py:198: UserWarning: data shape (19991, 15437) exceeds GL_MAX_TEXTURE_SIZE 16384 in at least one axis and will be downsampled. Rendering is currently in 2D mode.
  warnings.warn(
2025-01-16 09:35:32.827 Python[80492:117330978] TSM AdjustCapsLockLEDForKeyTransitionHandling - _ISSetPhysicalKeyboardCapsLockLED Inhibit


: 

In [8]:
exit()

: 